In [18]:
#One time run to import the project to your drive
!git clone https://github.com/KodeWil/final_project_AI_csv_Data.git

fatal: destination path 'final_project_AI_csv_Data' already exists and is not an empty directory.


In [0]:
def addData(files: str) -> list:
    allCsv = []
    for file in files:
      temp = open(file)
      tempDf = pd.read_csv(file)
      allCsv.append(tempDf)
    return allCsv

In [20]:
import pandas as pd 
import glob 
frames = []

#Create a list ('frames') with four nested lists, one per sensor. 
#Each nested list have all the samples (DF) for that sensor.
%cd /content/final_project_AI_csv_Data/phone/accel
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/phone/gyro
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/watch/accel
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/watch/gyro
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
# 0 in frames = /phone/accel
# 1 in frames = /phone/gyro
# 2 in frames = /watch/accel
# 3 in frames = /watch/gyro

/content/final_project_AI_csv_Data/phone/accel
/content/final_project_AI_csv_Data/phone/gyro
/content/final_project_AI_csv_Data/watch/accel
/content/final_project_AI_csv_Data/watch/gyro


In [0]:
import sys
from IPython.display import Image, display
import copy
import random
import pandas as pd
import numpy as np

In [0]:
def printAll(value):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(value)

In [0]:
def activityType(activity : str) -> int:
    noHand = ['A', 'B', 'C', 'M', 'E']
    eating = ['H', 'I', 'J', 'K', 'L']
    if (activity in noHand):
      return 0
    elif (activity in eating):
      return 1
    else:
      return 2


In [0]:
def mapActivitiesTemp(activities):
  mapped = activities
  map(activityType, mapped)

In [0]:
def mapActivities(activities):
  noHand = ['A', 'B', 'C', 'M', 'E']
  eating = ['H', 'I', 'J', 'K', 'L']
  mapped = activities
  for i in range(0,len(activities)):
    if (activities[i] in noHand):
      mapped[i] = 0
    elif (activities[i] in eating):
      mapped[i] = 1
    else:
      mapped[i] = 2
  return mapped

In [0]:
#Directly related features are wrapped in intervals on a list
def selectFeatures(features) -> list:
  labels = copy.deepcopy(features)
  labels = labels.tolist()
  intervals = []
  atts = []
  intervals.append([2, 31]) 
  intervals.append([32, 34]) # {X,Y,Z}AVG 
  intervals.append([35, 37]) # {X,Y,Z}PEAK 
  intervals.append([38, 40]) # {X,Y,Z}ABSOLDEV 
  intervals.append([41, 43]) # {X,Y,Z}STANDDEV 
  intervals.append([44, 46]) # {X,Y,Z}VAR* 
  intervals.append([47, 85]) # FCC
  intervals.append([86, 88]) # COS
  intervals.append([89, 91]) # COR
  intervals.append([92, 92]) # resultant
  selected = []
  nFeatures = random.randint(0, 1)
  for i in range (0, nFeatures + 1):
    selected.append(i)

  for element in selected:
    ini, fin = intervals[element]
    if (ini != 92):
      for i in range(ini, fin + 1):
        atts.append(features[i])
    else:
      atts.append(features[92])
  return atts


In [0]:
#Kmeans clustering
def scoreK(values, ground, clusters)->list:
  from sklearn.cluster import KMeans
  from sklearn import metrics
  # en n_clusters ponemos el número de actividades = 18
  kmeans = KMeans(n_clusters=clusters).fit(values)
  predicho = kmeans.predict(values)

  # adjusted_rand_score entre -1 y 1, mayor que cero es bueno, 1 es perfecto
  rand = metrics.adjusted_rand_score(ground, predicho)
  rand = round(rand, 4)
  
  # adjusted_mutual_info_score 0 y 1, 1 es perfecto
  mutual = metrics.adjusted_mutual_info_score(ground, predicho)
  mutual = round(mutual, 4)
  
  score = (rand, mutual)
  return score

In [0]:
#Agglomerative clustering
def scoreAgglo(values, ground, clusters)->list:
  from sklearn.cluster import AgglomerativeClustering
  from sklearn import metrics
  
  # en n_clusters ponemos el número de actividades = 18
  aglo = AgglomerativeClustering(n_clusters= clusters).fit(values)
  predicho = aglo.labels_

  # adjusted_rand_score entre -1 y 1, mayor que cero es bueno, 1 es perfecto
  rand = metrics.adjusted_rand_score(ground, predicho)
  rand = round(rand, 4)

  # adjusted_mutual_info_score 0 y 1, 1 es perfecto
  mutual = metrics.adjusted_mutual_info_score(ground, predicho)
  mutual = round(mutual, 4)

  score = (rand, mutual)
  return score

In [0]:
#Optics clustering
def scoreOptics(values, ground)->list:

  from sklearn.cluster import OPTICS
  from sklearn import metrics

  score = []
  # en n_clusters ponemos el número de actividades = 18
  optic = OPTICS(algorithm = 'auto').fit(values)
  predicho = optic.labels_

  # adjusted_rand_score entre -1 y 1, mayor que cero es bueno, 1 es perfecto
  rand = metrics.adjusted_rand_score(ground, predicho)
  rand = round(rand, 4)

  # adjusted_mutual_info_score 0 y 1, 1 es perfecto
  mutual = metrics.adjusted_mutual_info_score(ground, predicho)
  mutual = round(mutual, 4)
  score.append(rand)
  score.append(mutual)
  return score

In [0]:
#MiniBatchKMeans clustering

def ScoreMiniBatchK(values, ground, clusters)->list:

  from sklearn.cluster import MiniBatchKMeans
  from sklearn import metrics

  score = []
  # en n_clusters ponemos el número de actividades = 18
  miniBatch = MiniBatchKMeans(n_clusters = clusters).fit(values)
  predicho = miniBatch.predict(values)

  # adjusted_rand_score entre -1 y 1, mayor que cero es bueno, 1 es perfecto
  rand = metrics.adjusted_rand_score(ground, predicho)
  rand = round(rand, 4)

  # adjusted_mutual_info_score 0 y 1, 1 es perfecto
  mutual = metrics.adjusted_mutual_info_score(ground, predicho)
  mutual = round(mutual, 4)
  score.append(rand)
  score.append(mutual)
  return score

In [0]:
#Calculate adjusted_rand_score and mutual_info_score for each sample and each clustering method
def calcScore(sample)->list:
  
  #Set values to evalute clustering score
  ground_truth = sample["ACTIVITY"].values
  lookupTable, ground_truth = np.unique(ground_truth, return_inverse=True)
  features = selectFeatures(sample.columns)
  X = sample[features].values

  #Evaluate clustering method

  #Kmeans
  temRand,temMutual = scoreK(X, ground_truth, 18)
  scoresK = (temRand, temMutual)

  #Agglomerative
  temRand,temMutual = scoreAgglo(X, ground_truth, 18)
  scoresAgglo = (temRand, temMutual)

  #OPTICS
  temRand,temMutual = scoreOptics(X, ground_truth)
  scoresOpt = (temRand, temMutual)
  scores = [scoresK, scoresAgglo, scoresOpt]

  #Spectral
  temRand,temMutual = ScoreMiniBatchK(X, ground_truth, 18)
  scoresSpec = (temRand, temMutual)

  scores = [scoresK, scoresAgglo, scoresOpt, scoresSpec]
  return scores

In [0]:
#rank is a list that contains 50 list (one per sample), a those 50 lists have a list for clustering method.
# The most nested list contain the values for the two calculated scores
def sensorScores(samplesDf):
  rank = list(map(calcScore, samplesDf))
  scoreFrame = pd.DataFrame(rank)
  cluster = ['Kmeans', 'Agglomerative', 'OPTICS', 'MiniBatchKMeans']
  scoreFrame.columns = cluster
  return scoreFrame

In [0]:
allRanks = list(map(sensorScores, frames))


In [43]:
printAll(allRanks[0])

,Kmeans,Agglomerative,OPTICS,MiniBatchKMeans
0,"(0.354, 0.5575)","(0.3391, 0.5617)","(0.1058, 0.4481)","(0.3514, 0.5639)"
1,"(0.5783, 0.7755)","(0.5798, 0.7782)","(0.1986, 0.5794)","(0.5719, 0.7697)"
2,"(0.5398, 0.7591)","(0.542, 0.765)","(0.3022, 0.6608)","(0.5558, 0.7551)"
3,"(0.5399, 0.7605)","(0.542, 0.765)","(0.3022, 0.6608)","(0.5728, 0.7567)"
4,"(0.292, 0.5397)","(0.2416, 0.4976)","(0.0969, 0.4298)","(0.2471, 0.5217)"
5,"(0.2381, 0.5077)","(0.2416, 0.4976)","(0.0969, 0.4298)","(0.2429, 0.4996)"
6,"(0.5459, 0.7504)","(0.542, 0.765)","(0.3022, 0.6608)","(0.4979, 0.7343)"
7,"(0.2559, 0.5147)","(0.2416, 0.4976)","(0.0969, 0.4298)","(0.2797, 0.514)"
8,"(0.538, 0.7423)","(0.542, 0.765)","(0.3022, 0.6608)","(0.5365, 0.7541)"
9,"(0.6345, 0.7739)","(0.621, 0.7638)","(0.2695, 0.6057)","(0.5926, 0.7563)"


In [0]:
# Wrap activities in three categories as suggested in dataset description 
# seleccionar = ["ACTIVITY"]
# X = allData[seleccionar].values
# activityType = copy.deepcopy(X)
# activityType = mapActivities(activityType)
# allData.insert(0, "actType", activityType, True)